In [16]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! pip install mne

In [17]:
import pandas as pd
from glob import glob
import numpy as np
import os

import pywt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from google.cloud import storage
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix, classification_report
import time


## Functions to read data and load it

In [18]:


def prepare_data_train(fname):
    """ read and prepare training data """
    # Read data
    data = pd.read_csv(fname)
    # events file
    events_fname = fname.replace('_data','_events')
    # read event file
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels

In [19]:
def loading_one_subject(subject_number):
    y_raw= []
    raw = []
    fnames =  sorted(glob('/content/drive/MyDrive/train/subj%d_series*_data.csv' % (subject_number)))
    for fname in fnames:
        data,labels=prepare_data_train(fname)
        raw.append(data)
        y_raw.append(labels)
    X = pd.concat(raw).reset_index()
    X.drop(columns="index", inplace=True)
    y = pd.concat(y_raw).reset_index()
    y.drop(columns="index", inplace=True)
    return X, y

In [20]:
X, y = loading_one_subject(1) 

In [21]:
X.shape, y.shape


((1422392, 32), (1422392, 6))

In [22]:
type(X),type(y)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

## Function for mean absolute deviation of a signal and wavelet_denoising

In [23]:
def convert_df_to_numpy(df):
    return np.asarray(df.astype(float))
  

In [24]:
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [25]:
def wavelet_denoising(X, wavelet='db2', level=3):
    coeff = pywt.wavedec(X, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(X)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

In [26]:
def preprocess_data(X, y):
    X= convert_df_to_numpy(X)
    y=convert_df_to_numpy(y)
    X=wavelet_denoising(X)
    return X, y

In [27]:
X, y = preprocess_data(X,y)

In [28]:
X.shape, y.shape

((1422392, 32), (1422392, 6))

In [29]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

## Visualizing a signal before and after wavelet denoising

In [30]:
#signal=pd.read_csv('../raw_date/train/subj1_series1_data.csv')
#signal = signal.drop("id", axis=1)
#filtered = wavelet_denoising(signal, wavelet='db2', level=3)

#import matplotlib.pyplot as plt
#fig, ax = plt.subplots(figsize=(8,5))
#ax.plot(signal.iloc[:10000,1], label='signal', color="b", alpha=0.5,)
#ax.plot(filtered[:10000,1], label='reconstructed signal',color="k")
#ax.legend(loc='upper left')
#ax.set_title('Denoising with DWT')
#plt.show()

## Test splitting data

In [31]:
def custom_train_test_split(X, y):
     
    splitrate=-X.shape[0]//5*2
    xval=X[splitrate:splitrate//2]
    yval=y[splitrate:splitrate//2]
    xtest=X[splitrate//2:]
    ytest=y[splitrate//2:]
    xtrain=X[:splitrate]
    ytrain=y[:splitrate]
    
    return xtrain, xval, xtest, ytrain, yval, ytest 

In [32]:
# splitting the data 
xtrain, xval, xtest, ytrain, yval, ytest = custom_train_test_split(X, y)

In [33]:
xtrain.shape, xval.shape, xtest.shape, ytrain.shape, yval.shape, ytest.shape

((853434, 32),
 (284479, 32),
 (284479, 32),
 (853434, 6),
 (284479, 6),
 (284479, 6))

In [34]:
type(xtrain), type(xval), type(xtest), type(ytrain), type(yval), type(ytest)

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray)

## Scaling data

In [41]:
scaler= StandardScaler()

def data_scaler_train(X):
    X_prep=scaler.fit_transform(X)
    #do here your preprocessing
    return X_prep
def data_scaler_test(X):
    X_prep=scaler.transform(X)
    #do here your preprocessing
    return X_prep

In [42]:
X_train_scaled= data_scaler_train(xtrain)
X_val_scaled= data_scaler_test(xval)

In [43]:
X_train_scaled.shape, X_val_scaled.shape

((853434, 32), (284479, 32))

In [44]:
type(X_train_scaled), type(X_val_scaled)

(numpy.ndarray, numpy.ndarray)

## Generators

In [45]:
time_steps = 1000
subsample = 50

In [46]:
def generator(batch_size, xtrain, ytrain):
    while 1:
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtrain)-time_steps)
            x_time_data[i] = xtrain[random_index:random_index+time_steps:subsample]
            yy.append(ytrain[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1],  x_time_data.shape[2],1)), yy

In [47]:
def valgenerator(xval, yval):
    while 1:
        batch_size=32
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xval)-time_steps)
            x_time_data[i] = xval[random_index:random_index+time_steps:subsample]
            yy.append(yval[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2],1)), yy

# CNN Model 

In [49]:
def CNN_model():
  
  load = 1
  model = Sequential()

  model.add(Conv2D(filters = 64, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = ((time_steps//subsample), 32, 1)))
  model.add(BatchNormalization())
  #model.add(MaxPooling2D(pool_size = (3,3)))
  model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "same", activation = "relu", input_shape = ((time_steps//subsample), 32, 1)))
  model.add(BatchNormalization())
  #model.add(MaxPooling2D(pool_size = (3,3)))
  model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = ((time_steps//subsample), 32, 1)))
  model.add(BatchNormalization())
  #model.add(MaxPooling2D(pool_size = (3,3)))
  model.add(Flatten())
  #model.add(Dropout(0.2))
  model.add(Dense(32, activation = "relu"))
  model.add(BatchNormalization())
  # model.add(Dropout(0.2))
  model.add(Dense(6, activation = "sigmoid"))


  adam = Adam(lr = 0.0001)

  model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ['accuracy','mse'])

  return model

In [50]:
model= CNN_model()

In [51]:
es= EarlyStopping(patience=2)
history =model.fit_generator(generator(32,X_train_scaled, ytrain), steps_per_epoch = 600, epochs = 50,validation_data=valgenerator(X_val_scaled, yval),
                              validation_steps=200,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
600/600 [==============================] - 41s 19ms/step - loss: 0.6863 - accuracy: 0.2224 - mse: 0.2445 - val_loss: 0.5878 - val_accuracy: 0.3281 - val_mse: 0.1981
Epoch 2/50
600/600 [==============================] - 11s 18ms/step - loss: 0.5508 - accuracy: 0.2670 - mse: 0.1803 - val_loss: 0.4436 - val_accuracy: 0.3116 - val_mse: 0.1299
Epoch 3/50
600/600 [==============================] - 11s 18ms/step - loss: 0.3745 - accuracy: 0.2714 - mse: 0.1006 - val_loss: 0.2624 - val_accuracy: 0.2867 - val_mse: 0.0572
Epoch 4/50
600/600 [==============================] - 11s 18ms/step - loss: 0.2314 - accuracy: 0.2837 - mse: 0.0486 - val_loss: 0.1564 - val_accuracy: 0.2209 - val_mse: 0.0275
Epoch 5/50
600/600 [==============================] - 11s 18ms/step - loss: 0.1530 - accuracy: 0.3129 - mse: 0.0289 - val_loss: 0.1152 - val_accuracy: 0.3166 - val_mse: 0.0201
Epoch 6/50
600/600 [==============================] - 11s 18ms/step - loss: 0.1123 - accuracy: 0.3351 - mse: 0.0216 - va

In [ ]:
def plot_training_history(history):
    acc = history.history['accuracy']
    loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
 
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training acc',linewidth=2)
    plt.plot(epochs, val_acc, 'r--', label='Validation acc',linewidth=2)
    plt.title('Training  accuracy')
    plt.legend()
 
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'k', label='Validationloss ')
    plt.title('Training loss')
    plt.legend()
 
    plt.show()
plot_training_history(history)




In [ ]:
time_frame =1000
steps=50

In [ ]:
def test_generator(xtest, ytest):
    while 1:
        batch_size = 32
        x_time_data = np.zeros((batch_size, time_frame//steps, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtest)-time_frame)
            x_time_data[i] = xtest[random_index:random_index+time_frame:steps]
            yy.append(ytest[random_index + time_frame])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

In [235]:
#final generator for testing 
def batching(xtest, ytest):
  time_frame =1000
  steps=50  
  batch_size = int(len(xtest)/time_frame)
  
  x_time_data = np.zeros((batch_size, time_frame//steps, 32))
  index = 0
  yy = []

  for i in range(batch_size):

    x_time_data[i] = xtest[index:index+time_frame:steps]
            
    yy.append(ytest[index + time_frame -1])

    index+=time_frame

  yy = np.asarray(yy)

  return  x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

In [236]:
X_test_scaled = data_scaler_test(xtest)

In [237]:
x, y = batching(X_test_scaled[:284000], ytest[:284000])

In [239]:
len(y)

284

In [225]:
my_batch_x, my_batch_y= batching(X_test_scaled,ytest)

In [219]:
len(my_batch_x)

5

In [194]:
my_batch_predictions= model.predict(my_batch_x)

In [197]:
my_batch_first_pred= my_batch_predictions[0]

In [200]:
my_batch_first_pred_rounded = my_batch_first_pred.round()

In [201]:
np.where(my_batch_first_pred_rounded == 1)

(array([], dtype=int64),)

In [196]:
#we can now predict no movement

In [203]:
my_batch_movement= my_batch_predictions[1]

In [205]:
my_batch_mov_rounded = my_batch_movement.round()

In [207]:
mov_indexes = np.where(my_batch_mov_rounded ==1)

In [214]:
for i in mov_indexes[0]:
  print(i)

3


In [215]:
 labels = {0 : 'Handstart (HS)',
                   1 : "Grasping (GS)",
                   2 : "Lift (LT)",
                   3 : "Hold (HD)",
                   4 : "Replace (RP)",
                   5 : "Release (RL)"}


In [216]:
for i in mov_indexes[0]:
  print(labels[i])

Hold (HD)


In [ ]:
def preloaded_predictions():
  my_batch_x, my_batch_y= batching(X_test_scaled,ytest)

In [ ]:
#np.round(predictions).flatten().reshape(160000,6)

#np.asarray(y_test_).flatten().reshape(160000,6)
#l=["Handstart (HS)","Grasping (GS)","Lift (LT)","Hold (HD)","Replace (RP)","Release (RL)"]
#print(classification_report(np.asarray(y_test_).flatten().reshape(160000,6), np.round(predictions).flatten().reshape(160000,6), target_names=l))

Predic

In [172]:
def prediction_labels(gen_data):
    predicted_labels = []
    predictions=[]
    y_test_=[]

    for i in range(1):
      x_test, y_test = next(gen_data)
      while not 1 in y_test:
          x_test, y_test = next(gen_data)
      predicts= model.predict(x_test)
      predictions.append(predicts)
      y_test_.append(y_test)

    labels = {0 : 'Handstart (HS)',
                   1 : "Grasping (GS)",
                   2 : "Lift (LT)",
                   3 : "Hold (HD)",
                   4 : "Replace (RP)",
                   5 : "Release (RL)"}

    for pred in predictions[0]:
      list_to_np =np.asarray(np.round(pred))
      pos_indices = np.where(list_to_np == 1)[1]
      if len(pos_indices) ==0:
          predicted_labels.append('No Movement')
      else:
          list_labeled_mov = []
          for index in pos_indices:
              list_labeled_mov.append(labels[index])
          predicted_labels.append(list_labeled_mov)
    return predicted_labels, y_test_


In [3]:

#@title Get the segment, preprocess it, scale it, generator, feed it to the model, translate labels (make_prediction function)


In [ ]:
def grab(subj_num,):
  X ,y = loading_one_subject(subj_num)
  X, y = preprocess_data(X,y)
  xtrain, xval, xtest, ytrain, yval, ytest = custom_train_test_split(X, y)
  X_test_scaled = data_scaler_test(xtest)
  gen_data = testing_generator(xtest, ytest, 1000)

  


In [ ]:
#useless functions for now
def tester_generator(xtest, time_steps= 1000, subsample=50):
    while 1:
        batch_size = 1
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtest)-time_steps)
            x_time_data[i] = xtest[random_index:random_index+time_steps:subsample]
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1))



def prediction_test(df, num_test):
  predictions = []
  num_test = num_test
  for i in range(num_test):
      x_test= next(df)
      predict = model.predict(x_test)
      predictions.append(predict)
  l=["Handstart (HS)","Grasping (GS)","Lift (LT)","Hold (HD)","Replace (RP)","Release (RL)"]
  results=[]
  for elem in np.round(predictions):
    if elem[0][0] == 1:
       results.append(l[0])
    if elem[0][1] ==1:
      results.append(l[1])
    if elem[0][2] ==1:
      results.append(l[2])
    if elem[0][3] ==1:
      results.append(l[3])
    if elem[0][4] ==1:
      results.append(l[4])
    if elem[0][5] ==1:
      results.append(l[5])
  return np.reshape(results,(num_test,6))



def scores(gen_data, num_test):
  #scores = []
  predictions=[]
  x_test_=[]
  y_test_=[]
  num_test = num_test
  for i in range(num_test):
      x_test, y_test = next(gen_data)
      while not 1 in y_test:
          x_test, y_test = next(gen_data)
      #score = model.evaluate(x_test, y_test, verbose=0)
      predicts= model.predict(x_test)
      predictions.append(predicts)
      #scores.append(score[1])
      x_test_.append(x_test)
      y_test_.append(y_test)
  #scores = np.asarray(scores)
  #np.mean(scores),
  return  x_test_, y_test_ , predictions
  